In [1]:
import pandas as pd
from pandas import DataFrame
import sqlite3

In [2]:
conn = sqlite3.connect('./data/checking-logs.sqlite')

In [3]:
test_results = pd.io.sql.read_sql(''' SELECT time, AVG(diff) as avg_diff
FROM (
SELECT uid, labname, AVG((strftime('%s', first_commit_ts) - deadlines)/3600) as diff,
CASE
    WHEN ((strftime('%s', first_commit_ts)) - strftime('%s', first_view_ts))>0
        THEN 'after'
    ELSE 'before'
END AS time
FROM test
LEFT JOIN deadlines
ON labname = labs
WHERE labname NOT IN ('project1')
GROUP BY uid, time
)
WHERE uid IN
(
 SELECT uid FROM
(SELECT uid,

CASE
    WHEN ((strftime('%s', first_commit_ts)) - strftime('%s', first_view_ts))>0
        THEN 'after'
    ELSE 'before'
END AS time
FROM test
WHERE labname NOT IN ('project1')
GROUP BY uid, time)
GROUP BY uid
HAVING COUNT(uid)==2
)
GROUP BY time
''', conn)
test_results

,time,avg_diff
0,after,-99.523810
1,before,-66.047619


In [4]:
control_results = pd.io.sql.read_sql(''' SELECT time, AVG(diff) as avg_diff
FROM (
SELECT uid, labname, AVG((strftime('%s', first_commit_ts) - deadlines)/3600) as diff,
CASE
    WHEN ((strftime('%s', first_commit_ts)) - strftime('%s', first_view_ts))>0
        THEN 'after'
    ELSE 'before'
END AS time
FROM control
LEFT JOIN deadlines
ON labname = labs
WHERE labname NOT IN ('project1')
GROUP BY uid, time
)
WHERE uid IN
(
 SELECT uid FROM
(SELECT uid,
CASE
    WHEN ((strftime('%s', first_commit_ts)) - strftime('%s', first_view_ts))>0
        THEN 'after'
    ELSE 'before'
END AS time
FROM control
WHERE labname NOT IN ('project1')
GROUP BY uid, time)
GROUP BY uid
HAVING COUNT(uid)==2
)
GROUP BY time
''', conn)
control_results

,time,avg_diff
0,after,-99.322222
1,before,-98.033333


In [5]:
# Предположение оказалось верным и наличие страницы с новостной лентой влияет на поведение учащихся, 
# так как в тестовой группе есть различие, а в контрольной нет

In [6]:
conn.close()